<a href="https://colab.research.google.com/github/JDS289/jd2016_ads_2024/blob/main/playingAround.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using GPU seems to sometimes cause a CUDA error; using TPU seems to sometimes cause "Unknown crash".
CPU takes approximately 10x longer than GPU.


3B has said False to "Paris is the capital of France". Might want to use 8B instead. For now, I'll continue to use 3B for speed/simplicity. This is also justified for "reproducibility" reasons, though if accuracy is negligible then switch to a better model.

8B works with TPU? (sample size 1)
but still insanely slow - hasn't yet answered a single prompt (i think it may have gotten one or two, but i don't remember)

try 3B with a sys_prompt?

In [1]:
from google.colab import userdata
hf_token = userdata.get("huggingface_secret")

from huggingface_hub import login
login(token=hf_token)

In [2]:
DEFAULT_MODEL = ["meta-llama/Llama-3.1-8B-Instruct", "meta-llama/Llama-3.2-3B-Instruct"][1]

from typing import Optional
import os
import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
from tqdm.notebook import tqdm

#import warnings
#warnings.filterwarnings('ignore')


# Display first 500 characters of extracted text as preview


device = "cuda" if torch.cuda.is_available() else "cpu"

#import os
#os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


SYS_PROMPT = "" # experiment with this?


#Let's load in the model and start processing the text chunks

accelerator = Accelerator()

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map="balanced",
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
"""model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_MODEL,
    torch_dtype=torch.bfloat16,
    use_safetensors=True,
    device_map="auto",  # Or try 'balanced' or 'balanced_low_0' for multi-GPU
    offload_folder="offload",  # Specify a folder for offloading model weights
    offload_state_dict=True,   # Enable offloading the state dict to disk
    offload_buffers=True,
    output_hidden_states=True,
    return_dict_in_generate=True,
)
tokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)
model, tokenizer = accelerator.prepare(model, tokenizer)
model.generation_config.pad_token_id = tokenizer.eos_token_id"""

'model = AutoModelForCausalLM.from_pretrained(\n    DEFAULT_MODEL,\n    torch_dtype=torch.bfloat16,\n    use_safetensors=True,\n    device_map="auto",  # Or try \'balanced\' or \'balanced_low_0\' for multi-GPU\n    offload_folder="offload",  # Specify a folder for offloading model weights\n    offload_state_dict=True,   # Enable offloading the state dict to disk\n    offload_buffers=True,\n    output_hidden_states=True,\n    return_dict_in_generate=True,\n)\ntokenizer = AutoTokenizer.from_pretrained(DEFAULT_MODEL, use_safetensors=True)\nmodel, tokenizer = accelerator.prepare(model, tokenizer)\nmodel.generation_config.pad_token_id = tokenizer.eos_token_id'

In [6]:
model.generation_config.output_hidden_states = True
model.generation_config.return_dict_in_generate = True
# print(model.generation_config)

In [8]:
def process_chunk(text_chunk):
    """Process a chunk of text and return both input and output for verification"""
    conversation = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text_chunk},
    ]

    prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            temperature=0.1, # was 0.7 # maybe check this
            top_p=0.9,
            max_new_tokens=512
        )

    processed_text = tokenizer.decode(output.sequences[0], skip_special_tokens=True).strip()
    processed_text = processed_text[processed_text.index("assistant") + 11:]


    """
    for i in range(len(output.hidden_states)):
        total_shape = np.array(output.hidden_states[i][0].shape)
        for j in range(1, len(output.hidden_states[i])):
            total_shape += np.array(output.hidden_states[i][j].shape)
        print(total_shape)"""
    return processed_text, output.hidden_states

In [9]:
import requests

capitals = requests.get("https://raw.githubusercontent.com/JDS289/BaLD4LLM/refs/heads/main/country-list.csv")
capitals_dict = {}
for string in capitals.content.decode('utf-8').split('\n')[1 : -1]:
  _, country, _, city, _ = string.split('"')
  capitals_dict[country] = city

print(capitals_dict)


{'Afghanistan': 'Kabul', 'Albania': 'Tirana', 'Algeria': 'Algiers', 'Andorra': 'Andorra la Vella', 'Angola': 'Luanda', 'Antigua and Barbuda': "St. John's", 'Argentina': 'Buenos Aires', 'Armenia': 'Yerevan', 'Australia': 'Canberra', 'Austria': 'Vienna', 'Azerbaijan': 'Baku', 'Bahamas': 'Nassau', 'Bahrain': 'Manama', 'Bangladesh': 'Dhaka', 'Barbados': 'Bridgetown', 'Belarus': 'Minsk', 'Belgium': 'Brussels', 'Belize': 'Belmopan', 'Benin': 'Porto-Novo', 'Bhutan': 'Thimphu', 'Bosnia and Herzegovina': 'Sarajevo', 'Botswana': 'Gaborone', 'Brazil': 'Brasília', 'Brunei': 'Bandar Seri Begawan', 'Bulgaria': 'Sofia', 'Burkina Faso': 'Ouagadougou', 'Burundi': 'Bujumbura', 'Cambodia': 'Phnom Penh', 'Cameroon': 'Yaoundé', 'Canada': 'Ottawa', 'Cape Verde': 'Praia', 'Central African Republic': 'Bangui', 'Chad': "N'Djamena", 'Chile': 'Santiago', 'China': 'Beijing', 'Colombia': 'Bogotá', 'Comoros': 'Moroni', 'Costa Rica': 'San José', 'Croatia': 'Zagreb', 'Cuba': 'Havana', 'Cyprus': 'Nicosia', "Côte d'Ivo

In [11]:
# trying to use fewer words in the prompt, hopefully for faster execution

true_list = [f"Is {capital} the capital of {country}?" for country, capital in capitals_dict.items()]
fixed_order_capitals_dict = list(capitals_dict.items())
false_list = [f"Is {fixed_order_capitals_dict[i][1]} the capital of {fixed_order_capitals_dict[(i+1) % len(capitals_dict)][0]}?" for i in range(len(capitals_dict))]
print(true_list[:10], false_list[:10])

def print_ret(x):
  print(x[:2])
  return x

results = [print_ret((q,) + process_chunk(f'{q} (Yes/No)')) for q in tqdm(true_list[:20])]
#results_false = [print_ret((q,) + process_chunk(f'{q} (Yes/No)')) for q in tqdm(false_list[:20])]

statements = []
responses = []
hidden_statess = []

for s, r, h in results:
  statements.append(s)
  responses.append(r)
  hidden_statess.append(h)

['Is Kabul the capital of Afghanistan?', 'Is Tirana the capital of Albania?', 'Is Algiers the capital of Algeria?', 'Is Andorra la Vella the capital of Andorra?', 'Is Luanda the capital of Angola?', "Is St. John's the capital of Antigua and Barbuda?", 'Is Buenos Aires the capital of Argentina?', 'Is Yerevan the capital of Armenia?', 'Is Canberra the capital of Australia?', 'Is Vienna the capital of Austria?'] ['Is Kabul the capital of Albania?', 'Is Tirana the capital of Algeria?', 'Is Algiers the capital of Andorra?', 'Is Andorra la Vella the capital of Angola?', 'Is Luanda the capital of Antigua and Barbuda?', "Is St. John's the capital of Argentina?", 'Is Buenos Aires the capital of Armenia?', 'Is Yerevan the capital of Australia?', 'Is Canberra the capital of Austria?', 'Is Vienna the capital of Azerbaijan?']


  0%|          | 0/20 [00:00<?, ?it/s]

('Is Kabul the capital of Afghanistan?', 'Yes.')
('Is Tirana the capital of Albania?', 'Yes.')
('Is Algiers the capital of Algeria?', 'Yes')
('Is Andorra la Vella the capital of Andorra?', 'Yes.')
('Is Luanda the capital of Angola?', 'Yes')
("Is St. John's the capital of Antigua and Barbuda?", "No, St. John's is the capital of Antigua.")
('Is Buenos Aires the capital of Argentina?', 'Yes')
('Is Yerevan the capital of Armenia?', 'Yes.')
('Is Canberra the capital of Australia?', 'Yes.')
('Is Vienna the capital of Austria?', 'Yes')
('Is Baku the capital of Azerbaijan?', 'Yes')
('Is Nassau the capital of Bahamas?', 'Yes')
('Is Manama the capital of Bahrain?', 'Yes')
('Is Dhaka the capital of Bangladesh?', 'Yes.')
('Is Bridgetown the capital of Barbados?', 'Yes.')
('Is Minsk the capital of Belarus?', 'Yes.')
('Is Brussels the capital of Belgium?', 'Yes')
('Is Belmopan the capital of Belize?', 'No, Belmopan is not the capital of Belize. The capital of Belize is Belmopan is actually the secon

In [ ]:
"""true_list = [f"{capital} is the capital city of {country}" for country, capital in capitals_dict.items()]
fixed_order_capitals_dict = list(capitals_dict.items())
false_list = [f"{fixed_order_capitals_dict[i][1]} is the capital city of {fixed_order_capitals_dict[(i+1) % len(capitals_dict)][0]}" for i in range(len(capitals_dict))]
print(false_list[:20])

def print_ret(x):
  print(x[:2])
  return x

results = [print_ret((statement,) + process_chunk(f'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")')) for statement in tqdm(true_list[:20])]
results_false = [print_ret((statement,) + process_chunk(f'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")')) for statement in tqdm(false_list[:20])]

statements = []
responses = []
hidden_statess = []

for s, r, h in results:
  statements.append(s)
  responses.append(r)
  hidden_statess.append(h)
"""

'true_list = [f"{capital} is the capital city of {country}" for country, capital in capitals_dict.items()]\nfixed_order_capitals_dict = list(capitals_dict.items())\nfalse_list = [f"{fixed_order_capitals_dict[i][1]} is the capital city of {fixed_order_capitals_dict[(i+1) % len(capitals_dict)][0]}" for i in range(len(capitals_dict))]\nprint(false_list[:20])\n\ndef print_ret(x):\n  print(x[:2])\n  return x\n\nresults = [print_ret((statement,) + process_chunk(f\'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")\')) for statement in tqdm(true_list[:20])]\nresults_false = [print_ret((statement,) + process_chunk(f\'"{statement}". Is this true or false? (Answer ONLY with "True." or "False.")\')) for statement in tqdm(false_list[:20])]\n\nstatements = []\nresponses = []\nhidden_statess = []\n\nfor s, r, h in results:\n  statements.append(s)\n  responses.append(r)\n  hidden_statess.append(h)\n'

In [ ]:
process_chunk("hi")

('How can I assist you today?',
 ((tensor([[[-1.1587e-04,  3.8528e-04, -1.9379e-03,  ...,  2.3937e-04,
             -5.4550e-04,  8.8215e-05],
            [-1.1587e-04,  3.8528e-04, -1.9379e-03,  ...,  2.3937e-04,
             -5.4550e-04,  8.8215e-05],
            [-1.4160e-02,  1.2390e-02,  8.9264e-04,  ..., -1.6785e-03,
              6.3171e-03, -3.3722e-03],
            ...,
            [-6.5308e-03, -6.9809e-04, -6.7902e-04,  ...,  4.3869e-04,
              1.4038e-02,  1.0834e-03],
            [-3.7109e-02, -2.4872e-03,  3.2959e-02,  ..., -4.9805e-02,
              2.3438e-02,  1.3062e-02],
            [-1.5137e-02,  1.0071e-02, -4.9438e-03,  ...,  3.7689e-03,
             -1.1475e-02, -2.3346e-03]]], device='cuda:0', dtype=torch.bfloat16),
   tensor([[[-0.0031,  0.0019,  0.0601,  ...,  0.0510,  0.0021, -0.0049],
            [-0.0031,  0.0019,  0.0601,  ...,  0.0510,  0.0021, -0.0049],
            [-0.0303,  0.0046, -0.0664,  ...,  0.0121, -0.0248,  0.0044],
            ...,
    

In [ ]:
print(statements)

['Is Kabul the capital of Afghanistan?', 'Is Tirana the capital of Albania?', 'Is Algiers the capital of Algeria?', 'Is Andorra la Vella the capital of Andorra?', 'Is Luanda the capital of Angola?', "Is St. John's the capital of Antigua and Barbuda?", 'Is Buenos Aires the capital of Argentina?', 'Is Yerevan the capital of Armenia?', 'Is Canberra the capital of Australia?', 'Is Vienna the capital of Austria?', 'Is Baku the capital of Azerbaijan?', 'Is Nassau the capital of Bahamas?', 'Is Manama the capital of Bahrain?', 'Is Dhaka the capital of Bangladesh?', 'Is Bridgetown the capital of Barbados?', 'Is Minsk the capital of Belarus?', 'Is Brussels the capital of Belgium?', 'Is Belmopan the capital of Belize?', 'Is Porto-Novo the capital of Benin?', 'Is Thimphu the capital of Bhutan?']


In [ ]:
# print(len(hidden_statess[0]))
print(hidden_statess[0][0])



# 7 * 29 * 60 * 3072  for EACH hidden_statess[i]

# 7 layers * 29 batches * 60 tokens per input * 3072 activations

# idk why there are 29 batches

(tensor([[[-1.1587e-04,  3.8528e-04, -1.9379e-03,  ...,  2.3937e-04,
          -5.4550e-04,  8.8215e-05],
         [-1.1587e-04,  3.8528e-04, -1.9379e-03,  ...,  2.3937e-04,
          -5.4550e-04,  8.8215e-05],
         [-1.4160e-02,  1.2390e-02,  8.9264e-04,  ..., -1.6785e-03,
           6.3171e-03, -3.3722e-03],
         ...,
         [-1.5991e-02, -1.2512e-02, -5.9814e-03,  ...,  1.3184e-02,
          -8.1787e-03, -4.7302e-03],
         [-2.1484e-02, -4.3335e-03,  1.2573e-02,  ..., -4.3945e-03,
           2.0996e-02,  1.5320e-02],
         [-1.5137e-02,  1.0071e-02, -4.9438e-03,  ...,  3.7689e-03,
          -1.1475e-02, -2.3346e-03]]], device='cuda:0', dtype=torch.bfloat16), tensor([[[-0.0031,  0.0019,  0.0601,  ...,  0.0510,  0.0021, -0.0049],
         [-0.0031,  0.0019,  0.0601,  ...,  0.0510,  0.0021, -0.0049],
         [-0.0303,  0.0046, -0.0664,  ...,  0.0121, -0.0248,  0.0044],
         ...,
         [ 0.0078,  0.0134, -0.0913,  ..., -0.0033,  0.0140,  0.0089],
         [-0.01

In [ ]:
conversation = [
      {"role": "system", "content": SYS_PROMPT},
      {"role": "user", "content": f'"{statements[0]}". Is this true or false? (Answer ONLY with "True." or "False.")'},
]

prompt = tokenizer.apply_chat_template(conversation, tokenize=False)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [ ]:
print(inputs["input_ids"].shape)

torch.Size([1, 58])
